In [181]:
import numpy as np
import pandas as pd
import pickle
from sklearn import datasets # sklearn comes with some toy datasets to practise
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from matplotlib import pyplot
from sklearn.metrics import silhouette_score

In [182]:
## Initialize Spotify with user credentials
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import config
import pandas as pd

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=config.c_id, client_secret=config.c_se))

In [183]:
## Function to use the downloaded documents
def load(filename = "filename.pickle"): 
    try: 
        with open(filename, "rb") as f: 
            return pickle.load(f) 
        
    except FileNotFoundError: 
        print("File not found!") 


# Setting everything for running the model

# Spotify API


In [184]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=config.c_id, client_secret=config.c_se))

In [185]:
# Get the hotlist from the first document
hotlist = pd.read_csv('hot_list.csv')
hotlist


,Unnamed: 0,title,artist_name
0,0,We Don't Talk About Bruno,"Carolina Gaitan, Mauro Castillo, Adassa, Rhenz..."
1,1,Do We Have A Problem?,Nicki Minaj X Lil Baby
2,2,Easy On Me,Adele
3,3,Heat Waves,Glass Animals
4,4,Stay,The Kid LAROI & Justin Bieber
...,...,...,...
95,95,Iffy,Chris Brown
96,96,When I'm Gone,Alesso / Katy Perry
97,97,Fair Trade,Drake Featuring Travis Scott
98,98,Megan's Piano,Megan Thee Stallion


# Function to get user input

In [186]:
## Get user input 
song_input = input("Please enter a name of a song:")
if song_input in hotlist.values:
    print("See other recommended song:")
    print(hotlist.sample())
else:
    print("Unfortunately, the song is not in the hot list!")

Please enter a name of a song:We Don't Talk About Bruno
See other recommended song:
   Unnamed: 0          title  artist_name
5           5  Super Gremlin  Kodak Black


# Getting the features of the imput song

In [244]:
song_input = sp.search(q = song_input, limit = 1 , market ="GB")
song_input 

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=tracks&type=track&market=GB&offset=0&limit=1',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6TKOZZDd5uV5KnyC5G4MUt'},
       'href': 'https://api.spotify.com/v1/artists/6TKOZZDd5uV5KnyC5G4MUt',
       'id': '6TKOZZDd5uV5KnyC5G4MUt',
       'name': 'Smokey Robinson & The Miracles',
       'type': 'artist',
       'uri': 'spotify:artist:6TKOZZDd5uV5KnyC5G4MUt'}],
     'external_urls': {'spotify': 'https://open.spotify.com/album/2NT1Pc0MXlXORHLBkHyKQW'},
     'href': 'https://api.spotify.com/v1/albums/2NT1Pc0MXlXORHLBkHyKQW',
     'id': '2NT1Pc0MXlXORHLBkHyKQW',
     'images': [{'height': 640,
       'url': 'https://i.scdn.co/image/ab67616d0000b27396ebd8447b29307e31f95090',
       'width': 640},
      {'height': 300,
       'url': 'https://i.scdn.co/image/ab67616d00001e0296ebd8447b29307e31f95090',
       'width': 300},
      {'height': 64,
       'u

In [245]:
song_input['tracks']['items'][0]['id']

'6QyQmdvQ1ywNccYa0pwLNQ'

In [246]:
input_song_features = sp.audio_features(song_input['tracks']['items'][0]['id'])
input_song_features

[{'danceability': 0.502,
  'energy': 0.333,
  'key': 7,
  'loudness': -10.914,
  'mode': 1,
  'speechiness': 0.0264,
  'acousticness': 0.699,
  'instrumentalness': 0,
  'liveness': 0.222,
  'valence': 0.729,
  'tempo': 96.982,
  'type': 'audio_features',
  'id': '6QyQmdvQ1ywNccYa0pwLNQ',
  'uri': 'spotify:track:6QyQmdvQ1ywNccYa0pwLNQ',
  'track_href': 'https://api.spotify.com/v1/tracks/6QyQmdvQ1ywNccYa0pwLNQ',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6QyQmdvQ1ywNccYa0pwLNQ',
  'duration_ms': 174360,
  'time_signature': 4}]

In [247]:
## Get only the numerical data There are colums that are categorical which are not needed
input_song_features = pd.DataFrame(input_song_features)
input_song_features_num = input_song_features._get_numeric_data()

In [248]:
## Drop the columns which are not needed
input_song_features_num = input_song_features_num.drop(columns=["key","mode","time_signature"])
input_song_features_num

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,0.502,0.333,-10.914,0.0264,0.699,0,0.222,0.729,96.982,174360


In [249]:
model = load("scaler.pickle")

In [250]:
## Scale the numerical features
input_song_scaled = model.transform(input_song_features_num)

In [251]:
## We use the same features as the one with had with the list of tracks from Spotify
input_song_scaled = pd.DataFrame(input_song_scaled, columns = ["danceability", "energy", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness",
                          "valence", "tempo", "duration_ms"])
input_song_scaled

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,-0.908543,-0.923834,-0.265143,-0.710312,1.089329,-0.492715,0.418221,0.941212,-0.785425,-0.416092


In [252]:
model_kmeans = load("kmeans_4.pickle")

In [253]:
## Check to which cluster belongs the track
input_song_cluster = model_kmeans.predict(input_song_scaled)


In [254]:
input_song_cluster

array([6], dtype=int32)

In [255]:
model_kmeans

KMeans(random_state=1234)

In [256]:
## Get the table from the clustering notebook. Here are the features of the spotify track list
cluster_id = pd.read_csv('cluster_id.csv')
cluster_id

,Unnamed: 0,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,...,tempo.1,type,id,uri,track_href,analysis_url,duration_ms.1,time_signature,artists,song_title
0,0,0.918,0.609,-5.640,0.0791,0.09280,0.000000,0.1390,0.304,128.008,...,128.008,audio_features,0CAfXk7DXMnon4gLudAp7J,spotify:track:0CAfXk7DXMnon4gLudAp7J,https://api.spotify.com/v1/tracks/0CAfXk7DXMno...,https://api.spotify.com/v1/audio-analysis/0CAf...,231400,4,Flo Rida,Low (feat. T-Pain)
1,1,0.899,0.713,-2.752,0.3660,0.25500,0.000000,0.0708,0.777,90.051,...,90.051,audio_features,7iL6o9tox1zgHpKUfh9vuC,spotify:track:7iL6o9tox1zgHpKUfh9vuC,https://api.spotify.com/v1/tracks/7iL6o9tox1zg...,https://api.spotify.com/v1/audio-analysis/7iL6...,193467,4,50 Cent,In Da Club
2,2,0.957,0.737,-4.880,0.1150,0.21000,0.000000,0.0616,0.934,107.088,...,107.088,audio_features,5FLctoy4mpCxLJ1QPneTOV,spotify:track:5FLctoy4mpCxLJ1QPneTOV,https://api.spotify.com/v1/tracks/5FLctoy4mpCx...,https://api.spotify.com/v1/audio-analysis/5FLc...,228240,4,Nelly,Hot In Herre
3,3,0.664,0.758,-6.583,0.2100,0.00238,0.000000,0.0598,0.701,99.259,...,99.259,audio_features,0TwBtDAWpkpM3srywFVOV5,spotify:track:0TwBtDAWpkpM3srywFVOV5,https://api.spotify.com/v1/tracks/0TwBtDAWpkpM...,https://api.spotify.com/v1/audio-analysis/0TwB...,235933,4,Beyoncé,Crazy In Love (feat. Jay-Z)
4,4,0.768,0.864,-6.145,0.0324,0.10100,0.000000,0.1620,0.891,110.008,...,110.008,audio_features,1Je1IMUlBXcx1Fz0WE7oPT,spotify:track:1Je1IMUlBXcx1Fz0WE7oPT,https://api.spotify.com/v1/tracks/1Je1IMUlBXcx...,https://api.spotify.com/v1/audio-analysis/1Je1...,173027,4,Spice Girls,Wannabe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5083,5083,0.651,0.801,-7.340,0.0514,0.12300,0.000000,0.0825,0.776,133.769,...,133.769,audio_features,5jzma6gCzYtKB1DbEwFZKH,spotify:track:5jzma6gCzYtKB1DbEwFZKH,https://api.spotify.com/v1/tracks/5jzma6gCzYtK...,https://api.spotify.com/v1/audio-analysis/5jzm...,200480,4,The Clash,London Calling - Remastered
5084,5084,0.543,0.529,-9.717,0.0301,0.32600,0.002690,0.7090,0.746,85.126,...,85.126,audio_features,3KPwt1LBpt1jVSHz8GXERo,spotify:track:3KPwt1LBpt1jVSHz8GXERo,https://api.spotify.com/v1/tracks/3KPwt1LBpt1j...,https://api.spotify.com/v1/audio-analysis/3KPw...,313840,4,Bad Company,Feel like Makin' Love - 2015 Remaster
5085,5085,0.468,0.607,-11.367,0.0336,0.07200,0.000852,0.2250,0.886,98.429,...,98.429,audio_features,3MODES4TNtygekLl146Dxd,spotify:track:3MODES4TNtygekLl146Dxd,https://api.spotify.com/v1/tracks/3MODES4TNtyg...,https://api.spotify.com/v1/audio-analysis/3MOD...,263333,4,Led Zeppelin,Ramble On - 1990 Remaster
5086,5086,0.299,0.787,-7.930,0.0690,0.14000,0.000000,0.2920,0.716,94.798,...,94.798,audio_features,45s88Xopo6KvHc0PQ05aGg,spotify:track:45s88Xopo6KvHc0PQ05aGg,https://api.spotify.com/v1/tracks/45s88Xopo6Kv...,https://api.spotify.com/v1/audio-analysis/45s8...,198867,4,The Who,My Generation - Mono Version


In [257]:
## Drop the unnamed column
cluster_id.drop(columns=["Unnamed: 0"], inplace=True)
cluster_id

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,...,tempo.1,type,id,uri,track_href,analysis_url,duration_ms.1,time_signature,artists,song_title
0,0.918,0.609,-5.640,0.0791,0.09280,0.000000,0.1390,0.304,128.008,231400,...,128.008,audio_features,0CAfXk7DXMnon4gLudAp7J,spotify:track:0CAfXk7DXMnon4gLudAp7J,https://api.spotify.com/v1/tracks/0CAfXk7DXMno...,https://api.spotify.com/v1/audio-analysis/0CAf...,231400,4,Flo Rida,Low (feat. T-Pain)
1,0.899,0.713,-2.752,0.3660,0.25500,0.000000,0.0708,0.777,90.051,193467,...,90.051,audio_features,7iL6o9tox1zgHpKUfh9vuC,spotify:track:7iL6o9tox1zgHpKUfh9vuC,https://api.spotify.com/v1/tracks/7iL6o9tox1zg...,https://api.spotify.com/v1/audio-analysis/7iL6...,193467,4,50 Cent,In Da Club
2,0.957,0.737,-4.880,0.1150,0.21000,0.000000,0.0616,0.934,107.088,228240,...,107.088,audio_features,5FLctoy4mpCxLJ1QPneTOV,spotify:track:5FLctoy4mpCxLJ1QPneTOV,https://api.spotify.com/v1/tracks/5FLctoy4mpCx...,https://api.spotify.com/v1/audio-analysis/5FLc...,228240,4,Nelly,Hot In Herre
3,0.664,0.758,-6.583,0.2100,0.00238,0.000000,0.0598,0.701,99.259,235933,...,99.259,audio_features,0TwBtDAWpkpM3srywFVOV5,spotify:track:0TwBtDAWpkpM3srywFVOV5,https://api.spotify.com/v1/tracks/0TwBtDAWpkpM...,https://api.spotify.com/v1/audio-analysis/0TwB...,235933,4,Beyoncé,Crazy In Love (feat. Jay-Z)
4,0.768,0.864,-6.145,0.0324,0.10100,0.000000,0.1620,0.891,110.008,173027,...,110.008,audio_features,1Je1IMUlBXcx1Fz0WE7oPT,spotify:track:1Je1IMUlBXcx1Fz0WE7oPT,https://api.spotify.com/v1/tracks/1Je1IMUlBXcx...,https://api.spotify.com/v1/audio-analysis/1Je1...,173027,4,Spice Girls,Wannabe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5083,0.651,0.801,-7.340,0.0514,0.12300,0.000000,0.0825,0.776,133.769,200480,...,133.769,audio_features,5jzma6gCzYtKB1DbEwFZKH,spotify:track:5jzma6gCzYtKB1DbEwFZKH,https://api.spotify.com/v1/tracks/5jzma6gCzYtK...,https://api.spotify.com/v1/audio-analysis/5jzm...,200480,4,The Clash,London Calling - Remastered
5084,0.543,0.529,-9.717,0.0301,0.32600,0.002690,0.7090,0.746,85.126,313840,...,85.126,audio_features,3KPwt1LBpt1jVSHz8GXERo,spotify:track:3KPwt1LBpt1jVSHz8GXERo,https://api.spotify.com/v1/tracks/3KPwt1LBpt1j...,https://api.spotify.com/v1/audio-analysis/3KPw...,313840,4,Bad Company,Feel like Makin' Love - 2015 Remaster
5085,0.468,0.607,-11.367,0.0336,0.07200,0.000852,0.2250,0.886,98.429,263333,...,98.429,audio_features,3MODES4TNtygekLl146Dxd,spotify:track:3MODES4TNtygekLl146Dxd,https://api.spotify.com/v1/tracks/3MODES4TNtyg...,https://api.spotify.com/v1/audio-analysis/3MOD...,263333,4,Led Zeppelin,Ramble On - 1990 Remaster
5086,0.299,0.787,-7.930,0.0690,0.14000,0.000000,0.2920,0.716,94.798,198867,...,94.798,audio_features,45s88Xopo6KvHc0PQ05aGg,spotify:track:45s88Xopo6KvHc0PQ05aGg,https://api.spotify.com/v1/tracks/45s88Xopo6Kv...,https://api.spotify.com/v1/audio-analysis/45s8...,198867,4,The Who,My Generation - Mono Version


In [258]:
## Match the input song cluster with the clusters of the spotify list
match_song = cluster_id[cluster_id["cluster"] == int(input_song_cluster)]


In [259]:
## Return a random sample of items from an axis of object.
## Suggest a song that belongs to the same cluster
song_suggestion = match_song.sample()

In [260]:
## iloc gets rows (and/or columns) at integer locations.
## Sugggest the track id of the song that belongs to the same cluster
suggestion = song_suggestion["id"].iloc[0]
suggestion

'2Cc5z5swQrDqZiyCkZ0TnF'

In [261]:
## Function to get user input. It is a sum up of all the code above

def song_suggestion():
    
    song_input = input("Please enter a name of a song:")
    if song_input in hotlist.values:
        print("See other recommended song:")
        print(hotlist.sample())
    else:
        print("Unfortunately, the song is not in the hot list!")
        song_input = sp.search(q = song_input, limit = 1 , market ="GB")
        song_id = song_input["tracks"]["items"][0]["id"]
        input_song_features = sp.audio_features(song_input['tracks']['items'][0]['id'])
        input_song_features = pd.DataFrame(input_song_features)
        input_song_features_num = input_song_features._get_numeric_data()
        input_song_features_num = input_song_features_num.drop(columns=["key","mode","time_signature"])
        model = load("scaler.pickle")
        model_kmeans = load("kmeans_4.pickle")
        input_song_scaled = model.transform(input_song_features_num)
        input_song_scaled = pd.DataFrame(input_song_scaled, columns = ["danceability", "energy", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness",
                          "valence", "tempo", "duration_ms"])
       
        input_song_cluster = model_kmeans.predict(input_song_scaled)
        match_song = cluster_id[cluster_id["cluster"] == int(input_song_cluster)]
        song_suggestion = match_song.sample()
        suggestion = song_suggestion["id"].iloc[0]
        
        
    return IFrame(src="https://open.spotify.com/embed/track/"+suggestion,
               width="320",
               height="80",
               frameborder="0",
               allowtransparency="true",
               allow="encrypted-media",
                   )

    
    

In [262]:
song_suggestion()

Please enter a name of a song:crazy in love
Unfortunately, the song is not in the hot list!


In [ ]:
##song = sp.search(q=song_input,limit=1, market="GB")


In [237]:
## Get the song_id of the song_input to use in the next part of the code. (177)
song_id = song_input['tracks']['items'][0]['id']


In [235]:
from IPython.display import IFrame

#track_id = "1rfORa9iYmocEsnnZGMVC4"

IFrame(src="https://open.spotify.com/embed/track/"+song_id,
       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",
      )